# Hogg Family Homestead

This is a project for comparing neighbourhoods in Toronto. First, everyone knows West is best, and if you can afford it, stay south of Eglinton. Stay the hell outta Rexdale. 

In [5]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes #because I can't remember if I installed this already
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes # still can't remember
import folium 
from bs4 import BeautifulSoup
import lxml
print('All libraries imported.')

Solving environment: done

# All requested packages already installed.



AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

Look! A whole bunch of error messages that make no sense! I'm going to guess that libraries already installed means I did this already, so let's run with it and move on...


In [6]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

NameError: name 'BeautifulSoup' is not defined

Wait, you said all libraries were imported. Where's my BeautifulSoup?

In [7]:
from bs4 import BeautifulSoup

In [8]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [9]:
#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [10]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

In [11]:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

In [12]:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

In [13]:
# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

I feel compelled to point out at this point that East York was at one time "Canada's Only Borough" but Mike Harris ruined that. Thanks a lot, Mike!

In [15]:
# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

In [16]:
# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))


In [17]:
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))


In [19]:
# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

In [41]:
df2 = pd.DataFrame({'Postal code':df.Postal code.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postal code'] == x['Postal code']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postal code'] == x['Postal code']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

SyntaxError: invalid syntax (<ipython-input-41-15fdcd9762a0>, line 1)

This is driving me crazy! It seems that while the header is Postal code, that's not recognized as valid syntax. And why does the arrow point to the 'e' as being the problem. I would have thought the space, or a lack of a space, might be it. Anyway... I'll come back to this if I have to fix it.

In [46]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.12.0              |     pyhd8ed1ab_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium             conda-forge/noarch::folium-0.12.0-pyhd8ed1ab_0



folium-0.12.0        | 64 KB     | ##################################### | 100% 
branca-0.4.2         | 26 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction:

In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [50]:

List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [51]:
soup = BeautifulSoup(source, 'xml')

In [52]:
table=soup.find('table')

In [53]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood
column_names = ['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [54]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [55]:
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough              Neighbourhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

OK, so it looks like Postalcode all-one-word is the header here. At least I've figured that out. 

In [56]:
df=df[df['Borough']!='Not assigned']

In [58]:
df[df['Neighbourhood']=='Not assigned']=df['Borough']
df.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [60]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [61]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [62]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [63]:
df_merge.drop_duplicates(inplace=True)

In [64]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [65]:
df_merge.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [66]:
df_merge.shape

(103, 3)